## Introduction to Altair (part 2)

In this section, we will focus on including the interaction options in the Altair. 

Through interaction we can transform static images into tools for exploration: highlighting points of interest, zooming in to reveal finer-grained patterns, and linking across multiple views to reason about multi-dimensional relationships.

We will visualize the same car data in part 1 from the vega-datasets collection. 

In [1]:
# Install the module
import pandas as pd
import altair as alt

cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'

### Set parameters

We'll start with a scatter plot of horsepower versus miles per gallon, with a color encoding for the origin. We set the opacity as 0.5. 

In [2]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Horsepower:Q'),
    alt.Y('Miles_per_Gallon:Q'),
    alt.Color('Origin:N'),
    alt.OpacityValue(0.5)
)

alt.Chart(...)

The first thing we can do is to add a slide bar to change the settings in the figure, like the opacity. 

In [3]:
slider = alt.binding_range(min=0, max=1, step=0.05, name='opacity:')
op_var = alt.param(value=0.1, bind = slider)
#op_var = alt.param(value=0.1)

alt.Chart(cars).mark_circle().add_params(
    op_var
).encode(
    alt.X('Horsepower:Q'),
    alt.Y('Miles_per_Gallon:Q'),
    alt.Color('Origin:N'),
    alt.OpacityValue(op_var)
)

alt.Chart(...)

### Click and highlight

Maybe we don't want to change the opacity for all points. We jsut want to select part of the data to be highlighted. There are three types of selections in the Altair: 

- selection_point: select a single discrete value, by default on click events. You can also select multiple discrete values. The first value is selected on mouse click and additional values toggled using shift-click.
- selection_interval: select a continuous range of values, initiated by mouse drag.

In [4]:
#selection = alt.selection_point()
#selection = alt.selection_point(empty = False)
#selection = alt.selection_interval()
#selection = alt.selection_interval(encodings=['x'])
selection = alt.selection_point(on='mouseover')

alt.Chart(cars).mark_circle().add_params(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
)

alt.Chart(...)

### Select across multiple panels

This approach becomes even more powerful when the selection behavior is tied across multiple views of the data within a compound chart. 

In [5]:
selection = alt.selection_interval()

chart = alt.Chart(cars).mark_circle().encode(
    y='Horsepower:Q',
    x='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Origin:N', alt.value('lightgray'))
).add_params(
    selection
).properties(
    width = 200,
    height = 200
)

#chart
#chart.encode(x='Acceleration:Q')
chart | chart.encode(y='Acceleration:Q')


alt.HConcatChart(...)

Another thing we can do is to "zoom" the figure. We use the selection to get a filter of the data and make new figures. 

In [6]:
selection = alt.selection_interval()

points = alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N'
).add_params(
    selection
).properties(
    width = 200,
    height = 200
)

bars = alt.Chart(cars).mark_bar().encode(
    x='count()',
    y='Origin:N',
    color='Origin:N'
).transform_filter(
    selection
)

points & bars

alt.VConcatChart(...)

The interaction can also be added to the legend instead of the figure itself. In the following example, we can click the legend and only show part of the data in the figure. 

In [7]:
selection = alt.selection_point(fields = ['Origin'])
#selection = alt.selection_point(fields=['Origin', 'Cylinders'])

color = alt.condition(
    selection,
    alt.Color('Origin:N').legend(None),
    alt.value('lightgray')
)

scatter = alt.Chart(cars).mark_circle().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=color
)

#legend = alt.Chart(cars).mark_rect().encode(
legend = alt.Chart(cars).mark_circle().encode(
    alt.Y('Origin:N').axis(orient='right'),
#    alt.X('Cylinders:O'),
    color=color
).add_params(
    selection
)

scatter | legend


alt.HConcatChart(...)

### Bindings & Widgets

We can now bind parameters to chart elements (e.g. legends) and widgets (e.g. drop-downs and sliders) to allow users to do detailed screening on the data. 

- binding_checkbox: Renders as checkboxes allowing for multiple selections of items.
- binding_radio: Radio buttons that force only a single selection
- binding_select: Drop down box for selecting a single item from a list
- binding_range: Shown as a slider to allow for selection along a scale.

[Examples](https://altair-viz.github.io/gallery/multiple_interactions.html#gallery-multiple-interactions)

What we need to setup: 
- Set the dropdown (options, name etc. )
- Use selection to combine dropdown to the figure
- Make the figure

In [8]:
options = [None, 'Europe', 'Japan', 'USA']
labels = ['All', 'Europe', 'Japan', 'USA']

input_dropdown = alt.binding_select(options=options, 
                                    labels = labels,
                                    name='Region')

selection = alt.selection_point(fields=['Origin'], bind=input_dropdown)

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, alt.Color('Origin:N').legend(None),
                        alt.value('lightgray'))
).add_params(
    selection
)

alt.Chart(...)

In [9]:
options = ['Europe', 'Japan', 'USA']
labels = ['Europe', 'Japan', 'USA']

input_dropdown = alt.binding_radio(
    options=options + [None],
    labels=labels + ['All'],
    name='Region: '
)

selection = alt.selection_point(
    fields=['Origin'],
    bind=input_dropdown,
)

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    # We need to set a constant domain to preserve the colors
    # when only one region is shown at a time
    #color=alt.Color('Origin:N').scale(domain=options)
    color = alt.condition(selection, alt.Color('Origin:N').legend(None),
                        alt.value('lightgray'))
).add_params(
    selection
).transform_filter(
    selection
)

alt.Chart(...)

In [10]:
# Replicate example 1

slider = alt.binding_range(min=0, max=240, step=1, name='Cutoff ')
selection = alt.param(value=120, bind=slider)

chart1 = alt.Chart(cars).mark_point().encode(
   x='Horsepower:Q',
   y='Miles_per_Gallon:Q',
   color=alt.condition(
       alt.datum.Horsepower < selection,
       alt.value('red'),
       alt.value('blue')
   )
).add_params(
   selection
)

chart1

alt.Chart(...)

In [11]:
# Replicate example 2

shape_select = alt.binding_select(
    options=["arrow","circle","square","cross","diamond",
        "triangle","wedge","stroke"],
    name="shape",
)
shape_var = alt.param(bind=shape_select, value="circle")

angle_slider = alt.binding_range(min=-360, max=360, step=1, name="angle")
angle_var = alt.param(bind=angle_slider, value=0)

size_slider = alt.binding_range(min=0, max=500, step=10, name="size")
size_var = alt.param(bind=size_slider, value=50)

strokeWidth_slider = alt.binding_range(min=0, max=10, step=0.5, 
                                       name="strokeWidth")
strokeWidth_var = alt.param(bind=strokeWidth_slider, value=2)

chart = alt.Chart(cars).mark_point(
    shape=shape_var,
    angle=angle_var,
    size=size_var,
    strokeWidth=strokeWidth_var,
).encode(
    x="Horsepower:Q", 
    y="Miles_per_Gallon:Q"
).add_params(
    shape_var, angle_var, size_var, strokeWidth_var
)

chart

alt.Chart(...)

### Exercise:

Create a scatter plot that shows the relationship between Horsepower and Acceleration. Color the points based on the Origin of the cars. To do:

- Add a dynamic filtering option that allows users to filter the dataset based on the number of cylinders. 
- Create a second plot: a bar chart showing the number of cars by the number of cylinders.
- Add an interactive slider that allows users to filter the data based on the car's weight

In [12]:
chart = alt.Chart(cars).mark_point().encode(
    alt.X("Horsepower:Q"), 
    alt.Y("Acceleration:Q"),
    alt.Color('Origin:N')
)

In [13]:
filter_selection = alt.selection_point(
    fields=['Cylinders'], 
    bind=alt.binding_radio(options=[4, 6, 8])
)

# Apply the filter to the chart
filtered_chart = chart.add_params(
    filter_selection
).transform_filter(
    filter_selection
)

# Display the filtered chart
filtered_chart

alt.Chart(...)

In [14]:
selection = alt.selection_interval()

points = chart.add_params(
    selection
)

bars = alt.Chart(cars).mark_bar().encode(
    x='count()',
    y='Cylinders:N'
).transform_filter(
    selection
)

points | bars

alt.HConcatChart(...)

In [15]:
# Define a slider for filtering by car weight
weight_slider = alt.binding_range(min=1500, max=5000, step=100, name='Weight_in_lbs')
weight_selection = alt.param(value = 3000, bind=weight_slider)

# Filtered scatter plot with weight slider
scatter_plot_slider = chart.add_params(
    weight_selection
).transform_filter(
    alt.datum.Weight_in_lbs <= weight_selection
)

# Display the filtered scatter plot
scatter_plot_slider


alt.Chart(...)

### In-class activities: 

Pokemon again!
Now, take the three Altair plots you have made last time. Select one of the plots and include some interactions to it (beyond the tooltips this time!). Save the plot as an html file. 